<h1><b>Final Project Report</h1>

In [1]:
library(tidyverse)
library(tidymodels)


# loading player data into players df
players <- read_csv("data/players.csv")

# Filtering data for amateur players, ommiting missing data
filtered_players <- players |>
filter(experience == "Amateur") |>
na.omit()

filtered_players

# setting seed for project
set.seed(1)

# creating split & training/testing sets
player_split <- initial_split(filtered_players, strata = played_hours, prop = 0.75)
player_training <- training(player_split)
player_testing <- testing(player_split)


# creating workflow 

# recipe
knn_recipe <- recipe(played_hours ~ Age, data = filtered_players) |>
step_scale(all_predictors()) |>
step_center(all_predictors())

# model object
knn_spec <- nearest_neighbor(weight_func = "rectangular", mode = "regression", engine = "kknn", neighbors = tune()) 

# Cross-Validation

# creating validation sets
player_vfold <- vfold_cv(player_training, v = 5, strata = played_hours)
grid_vals <- tibble(neighbors = seq(1, 15, 1))

# creating tuning workflow
player_workflow <- workflow() |>
add_recipe(knn_recipe) |>
add_model(knn_spec)

player_results <- player_workflow |>
tune_grid(resamples = player_vfold, grid = grid_vals) |>
collect_metrics() |>
filter(.metric == "rmse")

# plot showing mean rmspe vs K value

result_plot <- player_results |>
ggplot(aes(x = neighbors, y = mean)) + 
geom_line() +
geom_point()

result_plot

best_k <- 4

player_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k, engine = "kknn", mode = "regression")

player_fit <- workflow() |>
add_model(player_spec) |>
add_recipe(knn_recipe) |>
fit(data = player_training)

player_summary <- player_fit |>
predict(player_testing) |>
bind_cols(player_testing) |>
metrics(truth = played_hours, estimate = .pred) |>
filter(.metric == 'rmse')

player_summary

pred_grid <- tibble(

    Age =  seq(

        from = filtered_players |> select(Age) |> min(),
        to = filtered_players |> select(Age) |> max(),
        by = 1
        
    )
)

player_preds <- player_fit |>
predict(pred_grid) |>
bind_cols(pred_grid)

player_plot <- ggplot(filtered_players, aes(x = Age, y = played_hours)) +
  geom_point(alpha = 0.4) +
  geom_line(data = player_preds,
            mapping = aes(x = Age, y = .pred),
            color = "steelblue",
            linewidth = 1) +
  xlab("Age of players (Years)") +
  ylab("Hours Played & Contributed") +
  ggtitle(paste0("K = ", best_k)) +
  theme(text = element_text(size = 12))

player_plot

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error: 'data/players.csv' does not exist in current working directory ('/home/jovyan/work/dsci-100-project-individual/Dsci100_Group_Project').
